In [1]:
import pandas as pd
import requests
from datetime import datetime, timedelta
import time

from urllib.parse import quote_plus

In [2]:
BOT_TOKEN = "5750455134:AAF-9R-v5a_qEMy_PK0vYb6oj2zbDu-VqIM"
API_KEY = "24VFZWRUPMEZ7E42477KPF7MRHUVBN1ZSX"
PAGE_SIZE = 1000
CHAT_ID = -1001591147163
PERIOD_HOURS = 12

In [3]:
def get_all_transactions(address: str, startblock: str):
    i = 1
    url_base = ('https://api.bscscan.com/api?' +
                'module=account' +
                '&action=txlist' +
                f'&address={address}' + 
                f'&apikey={API_KEY}' + 
                f'&startblock={startblock}' + 
                f'&offset={PAGE_SIZE}')
    all_transactions = []
    
    while True:
        url = url_base + f'&page={i}'
        print(f"Retrieving page #{i} of {address}")
        r = requests.get(url)
        transactions = r.json()["result"]
        all_transactions += transactions
    
        # if number of transactions is less then page maximum, then this is the last page and we can stop
        if len(transactions) < PAGE_SIZE: 
            return all_transactions
        
        # increase page number
        i += 1
        
        # wait a second to not spam the api
        time.sleep(1)
        

def filter_transactions_and_calculate_fees(transactions, onlyFrom: str):
    # make sure address is in lower case 
    onlyFrom = onlyFrom.lower() 
    
    return [
        {
            'hash': t['hash'],
            'datetime': datetime.fromtimestamp(int(t['timeStamp']), tz = None),
            'transactionFee': int(t['gasUsed']) * int(t['gasPrice']) * 1e-18,
            'functionName': t['functionName'],
            'isError': t['isError']
        } 
        for t in transactions 
        if t['from'] == onlyFrom 
    ]


def get_all_internal_transactions(address: str, startblock: str):
    i = 1
    url_base = ('https://api.bscscan.com/api?' +
                'module=account' +
                '&action=txlistinternal' +
                f'&address={address}' + 
                f'&apikey={API_KEY}' + 
                f'&startblock={startblock}' + 
                f'&offset={PAGE_SIZE}')
    all_transactions = []
    
    while True:
        url = url_base + f'&page={i}'
        print(f"Retrieving page #{i} of {address}")
        r = requests.get(url)
        transactions = r.json()["result"]
        all_transactions += transactions
    
        # if number of transactions is less then page maximum, then this is the last page and we can stop
        if len(transactions) < PAGE_SIZE: 
            return all_transactions
        
        # increase page number
        i += 1
        
        # wait a second to not spam the api
        time.sleep(1)
        

def calculate_fees_inner_transactions(transactions, filterFrom: str):
    # make sure address is in lower case 
    filterFrom = filterFrom.lower() 
    return [
        {
            'hash': t['hash'],
            'datetime': datetime.fromtimestamp(int(t['timeStamp']), tz = None),
            'value': int(t['value'])*1e-18,
            'isError': t['isError']
        } 
        for t in transactions 
        if t['from'] != filterFrom
    ]


def get_transaction_df(address: str, startblock: str):
    all_transactions = get_all_transactions(address, startblock)
    filtered = filter_transactions_and_calculate_fees(all_transactions, address)
    return pd.DataFrame(filtered).set_index('hash')

In [4]:
# load all transactions into dataframes
df_events = get_transaction_df('0xf3F4ffA90Bb740Af87F7BeD07551ce903cF89cd6', '23049523')
df_cron = get_transaction_df('0xD840708dE7dBAC08Df0CA1a79114778D78886165', '23049473')

TREASURY_ADDRESS = '0x869c420403B6db138BEe63d9646f4b8cDda8DF18'
internal_transactions = get_all_internal_transactions(TREASURY_ADDRESS, '23053796')
data = calculate_fees_inner_transactions(internal_transactions, TREASURY_ADDRESS)
df_treasury = pd.DataFrame(data).set_index('hash')

Retrieving page #1 of 0xf3F4ffA90Bb740Af87F7BeD07551ce903cF89cd6
Retrieving page #2 of 0xf3F4ffA90Bb740Af87F7BeD07551ce903cF89cd6
Retrieving page #3 of 0xf3F4ffA90Bb740Af87F7BeD07551ce903cF89cd6
Retrieving page #4 of 0xf3F4ffA90Bb740Af87F7BeD07551ce903cF89cd6
Retrieving page #5 of 0xf3F4ffA90Bb740Af87F7BeD07551ce903cF89cd6
Retrieving page #6 of 0xf3F4ffA90Bb740Af87F7BeD07551ce903cF89cd6
Retrieving page #7 of 0xf3F4ffA90Bb740Af87F7BeD07551ce903cF89cd6
Retrieving page #8 of 0xf3F4ffA90Bb740Af87F7BeD07551ce903cF89cd6
Retrieving page #9 of 0xf3F4ffA90Bb740Af87F7BeD07551ce903cF89cd6
Retrieving page #10 of 0xf3F4ffA90Bb740Af87F7BeD07551ce903cF89cd6
Retrieving page #1 of 0xD840708dE7dBAC08Df0CA1a79114778D78886165
Retrieving page #2 of 0xD840708dE7dBAC08Df0CA1a79114778D78886165
Retrieving page #3 of 0xD840708dE7dBAC08Df0CA1a79114778D78886165
Retrieving page #4 of 0xD840708dE7dBAC08Df0CA1a79114778D78886165
Retrieving page #1 of 0x869c420403B6db138BEe63d9646f4b8cDda8DF18
Retrieving page #2 of 0x

In [5]:
def create_report_body(df_events_, df_cron_, df_treasury_):
    # aggregate by function name
    g_events = df_events_.groupby('functionName')['transactionFee'].agg(['sum', 'count'])
    g_cron = df_cron_.groupby('functionName')['transactionFee'].agg(['sum', 'count'])
    df_agg = pd.DataFrame(g_cron.add(g_events, fill_value=0))
    df_agg.index = df_agg.index.map(lambda s: s.split('(')[0])
    df_agg.index.name = ''
    df_agg['count'] = df_agg['count'].astype(int)
    df_agg.columns = ["Fee", "Count"]
    
    # create a 'report' as a string that will be sent as telegram message
    total_fee_events = df_events_.transactionFee.sum()
    total_fee_cron = df_cron_.transactionFee.sum()
    treasury_sum = df_treasury_.value.sum()

    report = (
        f"Total fee, events: {total_fee_events:.3f} BNB\n" +
        f"Total fee, cron:     {total_fee_cron:.3f} BNB\n" +
        f"Collected from users:  {treasury_sum:.3f} BNB\n" + 
        f"Balance:               {treasury_sum - total_fee_events - total_fee_cron:.3f} BNB")

    report += "\n\n\n<b>Transaction fee by type (events + cron)</b>:\n\n<pre>" + df_agg.to_markdown() + "</pre>"


    failed_events = df_events_.isError.astype(int).sum()
    failed_cron = df_cron_.isError.astype(int).sum()

    report += (
        "\n\n\n<b>Transaction counts</b>:\n" + 
        f"\nTotal:   {len(df_events_) + len(df_cron_)}\n" +
        f"Failed:  {failed_events + failed_cron}\n" +
        f"\nEvents:  {len(df_events_)}\n" +
        f"Failed: {failed_events}\n" +
        f"\nCron:    {len(df_cron_)}\n" + 
        f"Failed:   {failed_cron}\n")
    return report

In [6]:
def get_last_period(df):
    return df[df.datetime > datetime.today() - timedelta(hours=PERIOD_HOURS)]

In [7]:
# sent report in 2 messages
report = "<b>Based on full history</b>:\n\n"
report += create_report_body(df_events, df_cron, df_treasury)
message = quote_plus(report)
r = requests.get(f"https://api.telegram.org/bot{BOT_TOKEN}/sendMessage?chat_id={CHAT_ID}&parse_mode=HTML&text={message}")

report = f"<b>Based on last {PERIOD_HOURS} hours</b>:\n\n"
report += create_report_body(get_last_period(df_events), get_last_period(df_cron), get_last_period(df_treasury))
message = quote_plus(report)
r = requests.get(f"https://api.telegram.org/bot{BOT_TOKEN}/sendMessage?chat_id={CHAT_ID}&parse_mode=HTML&text={message}")

In [8]:
r.json()

{'ok': True,
 'result': {'message_id': 74,
  'from': {'id': 5750455134,
   'is_bot': True,
   'first_name': 'Axes Transactions Fee Reporter',
   'username': 'axes_transactions_fee_bot'},
  'chat': {'id': -1001591147163,
   'title': 'Axes Transactions Fee Reporting',
   'type': 'supergroup'},
  'date': 1675677039,
  'text': 'Based on last 12 hours:\n\nTotal fee, events: 0.000 BNB\nTotal fee, cron:     0.391 BNB\nCollected from users:  0.000 BNB\nBalance:               -0.391 BNB\n\n\nTransaction fee by type (events + cron):\n\n|                  |       Fee |   Count |\n|:-----------------|----------:|--------:|\n| setUpgradeNewIds | 0.0103311 |      15 |\n| systemMint       | 0.380401  |      60 |\n\n\nTransaction counts:\n\nTotal:   75\nFailed:  0\n\nEvents:  0\nFailed: 0\n\nCron:    75\nFailed:   0',
  'entities': [{'offset': 0, 'length': 22, 'type': 'bold'},
   {'offset': 154, 'length': 39, 'type': 'bold'},
   {'offset': 196, 'length': 171, 'type': 'pre'},
   {'offset': 370, 'length